# Level 2 - Week 3 - 02 Filters and Top-K Tradeoffs

**Estimated time:** 60-90 minutes

## Learning Objectives

- Compare precision and recall tradeoffs
- Understand filter impact
- Log retrieval settings


## Overview

Filters improve precision, top_k improves recall.
You need to choose a tradeoff based on failures.

## Practice Steps

- Compute precision and recall for toy results.
- Compare two top_k settings.


### Sample code

Toy precision and recall calculation.


In [ ]:
def precision_at_k(relevant: set[str], retrieved: list[str]) -> float:
    if not retrieved:
        return 0.0
    hits = [r for r in retrieved if r in relevant]
    return len(hits) / len(retrieved)


def recall_at_k(relevant: set[str], retrieved: list[str]) -> float:
    if not relevant:
        return 0.0
    hits = [r for r in retrieved if r in relevant]
    return len(hits) / len(relevant)

relevant = {'A', 'B', 'C', 'D'}
retrieved_3 = ['A', 'B', 'X']
retrieved_8 = ['A', 'B', 'X', 'Y', 'C', 'Z', 'W', 'V']

print('p@3', precision_at_k(relevant, retrieved_3))
print('r@3', recall_at_k(relevant, retrieved_3))
print('p@8', precision_at_k(relevant, retrieved_8))
print('r@8', recall_at_k(relevant, retrieved_8))


### Student fill-in

Try a different filter or top_k and record the tradeoff.


In [ ]:
# TODO: change retrieved lists and observe precision/recall


## Self-check

- Do you log filters and top_k per request?
- Can you explain precision vs recall tradeoffs?
